In [1]:
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 645kB 8.5MB/s 
     |████████████████████████████████| 1.1MB 24.7MB/s 
     |████████████████████████████████| 3.8MB 53.2MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0f7f3ba3555d09ef8a5bc0cc0af81fc81bebe23da7ef244408579b08dba7d3bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import logging
import torch
import numpy as np
from tqdm import tqdm, trange
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import Dataset
from torch.nn.utils.rnn import pad_sequence
from typing import Dict, List, Tuple
from torch.utils.tensorboard.writer import SummaryWriter

In [3]:
!wget -nc -O ds.npy https://ibm.box.com/shared/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy

--2020-05-18 01:40:02--  https://ibm.box.com/shared/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy [following]
--2020-05-18 01:40:03--  https://ibm.box.com/public/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy [following]
--2020-05-18 01:40:03--  https://ibm.ent.box.com/public/static/o9x8cglra6bpvngt537nlo7pe7ctjrk6.npy
Resolving ibm.ent.box.com (ibm.ent.box.com)... 185.235.236.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|185.235.236.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d

In [7]:
logging.basicConfig(level=logging.INFO)
tb_writer = SummaryWriter()
EVALUATE = True

LANGUAGES = [
    "tensorflow",
    "pytorch"
]
tokenizer = RobertaTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
model = RobertaForSequenceClassification.from_pretrained("huggingface/CodeBERTa-small-v1", num_labels=len(LANGUAGES))

INFO:transformers.tokenization_utils:Model name 'huggingface/CodeBERTa-small-v1' not found in model shortcut name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). Assuming 'huggingface/CodeBERTa-small-v1' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/huggingface/CodeBERTa-small-v1/vocab.json from cache at /root/.cache/torch/transformers/4378e0cbd4e198a29d18acffb589430ff433fbbfd637803fddbc8e4b5108368f.516f21a3e3c538a6e0b52aea5987651b995c82fba6021ab1cd79ef987877432b
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/huggingface/CodeBERTa-small-v1/merges.txt from cache at /root/.cache/torch/transformers/2b5cb186465fac4f5d94ce950b4b26ba8333b9387ec72e11872819344c529f7e.cdee6ff5de424ca98962369a559ecdef682286d4dfc7a37d91f

INFO:transformers.file_utils:storing https://cdn.huggingface.co/huggingface/CodeBERTa-small-v1/pytorch_model.bin in cache at /root/.cache/torch/transformers/8df1efcdb462fb48d8e3d93d8f869641fce29d289d95b8d357a5854c7dc2b1b9.5561c147dbc48bf85cff1b96abd42419658212abeb129931ca149d3ad567a7d4
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/8df1efcdb462fb48d8e3d93d8f869641fce29d289d95b8d357a5854c7dc2b1b9.5561c147dbc48bf85cff1b96abd42419658212abeb129931ca149d3ad567a7d4
INFO:filelock:Lock 140278814953936 released on /root/.cache/torch/transformers/8df1efcdb462fb48d8e3d93d8f869641fce29d289d95b8d357a5854c7dc2b1b9.5561c147dbc48bf85cff1b96abd42419658212abeb129931ca149d3ad567a7d4.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/huggingface/CodeBERTa-small-v1/pytorch_model.bin from cache at /root/.cache/torch/transformers/8df1efcdb462fb48d8e3d93d8f869641fce29d289d95b8d357a5854c7dc2b1b9.5561c147dbc48bf85cff1b96abd42419658212ab

INFO:transformers.modeling_utils:Weights of RobertaForSequenceClassification not initialized from pretrained model: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
INFO:transformers.modeling_utils:Weights from pretrained model not used in RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']


In [0]:
class CodeSearchNetDataset(Dataset):
    examples: List[Tuple[List[int], int]]

    def __init__(self, data):
        """
        train or test data
        """

        self.examples = []

        lines = []
        for code in tqdm(data):
            if code[0] == '':
                continue
            label = code[1]
            label_idx = LANGUAGES.index(label)
            examples = [(tokenizer.encode(code[0], max_length=512), label_idx)]
            self.examples += examples
        print("🔥🔥")
        
        

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return self.examples[i]

In [0]:
ds = np.load('ds.npy', allow_pickle = True)
dstrain = ds[:int(len(ds)*.88)]
dstest = ds[int(len(ds)*.88):]

In [10]:
train_dataset = CodeSearchNetDataset(dstrain)
test_dataset = CodeSearchNetDataset(dstest)

  0%|          | 5/1447 [00:00<00:47, 30.50it/s]

🔥🔥


100%|██████████| 1447/1447 [00:22<00:00, 65.75it/s]

🔥🔥


In [11]:
def collate(examples):
    input_ids = pad_sequence([torch.tensor(x[0]) for x in examples], batch_first=True, padding_value=1)
    labels = torch.tensor([x[1] for x in examples])
    return input_ids, labels

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate)

batch = next(iter(train_dataloader))

model.to("cuda")
model.train()
for param in model.roberta.parameters():
    param.requires_grad = False
## ^^ Only train final layer.

print(f"num params:", model.num_parameters())
print(f"num trainable params:", model.num_parameters(only_trainable=True))

num params: 84043010
num trainable params: 592130


In [12]:
def evaluate():
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = np.empty((0), dtype=np.int64)
    out_label_ids = np.empty((0), dtype=np.int64)

    model.eval()

    eval_dataloader = DataLoader(test_dataset, batch_size=512, collate_fn=collate)
    for step, (input_ids, labels) in enumerate(tqdm(eval_dataloader, desc="Eval")):
        with torch.no_grad():
            outputs = model(input_ids=input_ids.to("cuda"), labels=labels.to("cuda"))
            loss = outputs[0]
            logits = outputs[1]
            eval_loss += loss.mean().item()
            nb_eval_steps += 1
        preds = np.append(preds, logits.argmax(dim=1).detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
    eval_loss = eval_loss / nb_eval_steps
    acc = simple_accuracy(preds, out_label_ids)
    f1 = f1_score(y_true=out_label_ids, y_pred=preds, average="macro")
    print("=== Eval: loss ===", eval_loss)
    print("=== Eval: acc. ===", acc)
    print("=== Eval: f1 ===", f1)
    # print(acc_and_f1(preds, out_label_ids))
    tb_writer.add_scalars("eval", {"loss": eval_loss, "acc": acc, "f1": f1}, global_step)


### Training loop

global_step = 0
train_iterator = trange(0, 4, desc="Epoch")
optimizer = torch.optim.AdamW(model.parameters())
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
    for step, (input_ids, labels) in enumerate(epoch_iterator):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.to("cuda"), labels=labels.to("cuda"))
        loss = outputs[0]
        loss.backward()
        tb_writer.add_scalar("training_loss", loss.item(), global_step)
        optimizer.step()
        global_step += 1
        if EVALUATE and global_step % 50 == 0:
            evaluate()
            model.train()


evaluate()

Iteration:  17%|█▋        | 7/42 [00:43<03:39,  6.28s/it]

Eval:   0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: ignored